# Accretion disk

In this tutorial we will set up an accretion disk, which is a radiation source separate from the star, and add its contribution to the signal. The accretion disk we will use here (Mitsuda et al. 1984, Makishima et al. 1986) is a superposition of multicolor blackbody rings. 

We follow here the description given in Makishima et al. 1986. The disk is parametrized by an inner cut-off radius, $R_\rm{in}$, a temperature at this radius, $T_\rm{in}$, and an outer radius, $R_\rm{out}$. The outer rings typically contribute much less to the overal spectrum than inner rings due to being colder, so the exact value of $R_\rm{out}$ is much less consequential than $R_\rm{in}$. For each ring, the temperature at some radius $T(r)$ is determined by the energy release associated with accretion towards a central object of mass $M_*$. It is expressed as:

$$T(r) = \left(\frac{3 G \dot{M} M_x}{8 \pi \sigma r^3}\right)^{1/4}.$$

Where $G$ is the gravitational constant, $\dot{M}$ is the accretion rate and $\sigma$ is the Stefan-Boltzmann constant. The specific luminosity (energy/time/energy) from both sides of one ring between $r-dr$ and $r$ is given by 

$$L_\rm{ring}(E) = 8 \pi r B(E,T) dr \int d\Omega = 8 \pi^2 B(E,T) r dr = 4\pi\sigma T^4 r dr.$$ 

we obtained a factor of $\pi$ from the integral over the solid angle. One factor of 2 is associated with the two sides of the disk and another factor of 2 appears since we define the $B(E, T)$, which is the blackbody spectral radiance, as 

$$B(E,T) = \frac{E^3}{c^2h^2}\left[\exp{\left(\frac{E}{k_\rm{B}T}\right)-1}\right]^{-1}, $$

which is missing the factor of 2 you typically see in contemporary literature. The specific flux of the disk $f_\rm{disk}(E)$ (energy/time/length$^2$/energy) is the integral of the contribution of each ring from $R_\rm{in}$ to $R_\rm{out}$. But, this can be conveniently expressed as an integral over $dT$:

$$f_\rm{disk}(E) = \frac{2 \pi \cos{i}}{D^2}\int_{R_\rm{in}}^{R_\rm{out}} r B(E,T(r)) dr = \frac{8\pi R_\rm{in}^2 \cos{i}}{3 D^2 T_\rm{in}}\int_{T_\rm{out}}^{T_\rm{in}}\Big(\frac{T}{T_\rm{in}}\Big)^{-11/3}B(E,T) dT.$$

Here we applied $F = L/(4\pi D^2)$ to get the flux at distance $D$ and we introduced $\cos{i}$ to scale the flux to account for the viewing angle. Finally, to obtain the bolometric flux (energy/length^2/time) of the disk one needs to integrate over energy:

$$F_\rm{disk} = \int f_\rm{disk} dE.$$

To obtain the photon flux for the disk instead of energy flux, one can repeat this process but with $b(E,T)$ instead of $B(E,T)$, which is the blackbody spectral radiance per photon energy: $b(E, T) = B(E,T)/E$. 